# Simulation #1
This workbook is a first attempt at constructing a simulation of a stochastic dynamical system, with an accompanying estimation of the Perron-Frobenius operator.

In [65]:
using LinearAlgebra, Plots, Distributions

In [66]:
include("functions.jl");
include("nnlsq.jl");

In [67]:
#pyplot();

## Setting up the dynamics
The state space $X$ of our dynamical system will be the flat torus $\mathbb{T}^2$, represented by the unit square.

In [68]:
range = 2π;
grid_size = 100;
n_gridpoints = grid_size ^ 2;

println("Our reference grid contains $n_gridpoints points.")

Our reference grid contains 10000 points.


In [69]:
grid = creategrid(0, range, grid_size);

We will define an initial density $f_0 (x, y)$ on this state space.

In [70]:
f0s = [f0(grid[n,:], range; dist="normal") for n in 1:n_gridpoints];
#surface(grid[:,1], grid[:,2], f0s, markersize=1)

To obtain a sample from this initial density $f_0$, we can use rejection sampling.

In [71]:
n_samples = 10000;
candidate_points = range .* rand(n_samples,2);
f0s = [f0(candidate_points[n,:], range; dist="normal") for n in 1:n_samples];
thresholds = 1/40 .* rand(n_samples);
sample = candidate_points[f0s .> thresholds, :];

#histogram2d(sample[:,1], sample[:,2], bins=20)

In [72]:
sample_size = size(sample, 1);
println("We obtain an initial sample of $sample_size points")

We obtain an initial sample of 4542 points


The particular map $S: \mathbb{T}^2 \to \mathbb{T}^2$ chosen to introduce the dynamics is the standard map, given by
$$
S ([x , y]) := \begin{bmatrix} x + y \\ y + a \sin ( x + y) \end{bmatrix} \mod 2\pi
$$
where $a$ is a parameter which controls the amount of chaos, and which for the moment we take to be $a=6$. This map is easily wrapped in a function ```S``` which can then be iterated forward across timesteps via ```Sforward```.

We can then easily compare the initial density $f_0$ with a naive estimate of a limiting density $f_*$ by computing the latter with a big value for ```S_forward```.

In [73]:
fstar = S_forward(sample, S, 100);

In [74]:
#histogram2d(fstar[:,1], fstar[:,2], bins=30)

This map $S$ is volume preserving, so we should expect to see the same density no matter how many times $S$ is applied.

## Estimating the Perron-Frobenius operator
Having established a model for the dynamics, next we want to estimate the Perron-Frobenius operator. 
$$
\mathcal{P} : L^1(X) \to L^1(X)
$$
To do so, we will need to work in a finite-dimensional subspace of $L^1(X)$. And actually. we will be estimating a stochastically perturbed version of $\mathcal{P}$, described by
$$
 \mathcal{L} f(y) = \int_X k(Sx, y ) f(x) dx
$$
where $k(x, y) = \phi(x-y)$. So our goal is an estimated matrix $L$.

To estimate $L$, we first must define this finite basis for $L^1(X)$. For the moment we will take this basis to be a uniform grid of radial basis functions (RBFs) $\varphi_i$. These will be of the form
$$
\varphi_i (x) = \phi ( x - z_i ) := \exp \left( - \frac{\| x - z_i \|^2}{\epsilon^2} \right)
$$
where the $z_i$ denote the centres of each RBF and $\epsilon$ is some bandwidth parameter. We can wrap this calculation in the general function ```\Phi```.

In [75]:
ϵ = 1;
c = π * ϵ^2;

In [76]:
basis_grid_size = 25;
n_bases = basis_grid_size ^ 2;

println("We choose a basis of $n_bases RBFs.")

We choose a basis of 625 RBFs.


In [77]:
basis_locs = creategrid(0, range, basis_grid_size);

These basis functions can be easily visualised by evaluating all grid points against each.

In [78]:
basis_evaluation_matrix = zeros(n_gridpoints, n_bases);
for b in 1:n_bases
    for i in 1:n_gridpoints
        basis_evaluation_matrix[i, b] = φ(grid[i, :], basis_locs[b, :], ϵ);
    end
end

In [79]:
basis_surface = sum(evaluation_matrix, dims=2);
#surface(grid[:,1], grid[:,2], basis_surface)

LoadError: UndefVarError: evaluation_matrix not defined

In order to estimate the integral
$$
    \mathcal{L} f(y) = \int_X k (Sx, y) f(x) dx
$$
we will also need an approximate Lebesgue measure for the integral wrt $dx$. This will be some weighted combination of all the datapoints, subject to the constraint that obviously each basis function should integrate to a constant. We can estimate these weights $w$ using nonnegative least squares, though this will first require an evaluation matrix on the sample according to the basis functions.

In [80]:
Φ = evaluate_phi(sample, basis_locs, φ, ϵ);

In [81]:
C = c * ones(n_bases);

In [82]:
w, residual, objvalue = nnlsq(Φ, C, 0);

Academic license - for non-commercial use only - expires 2021-08-05
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 625 rows, 5167 columns and 2756168 nonzeros
Model fingerprint: 0x54c3d429
Model has 625 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-13, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 3e+00]
Presolve time: 0.70s
Presolved: 625 rows, 5167 columns, 2756168 nonzeros
Presolved model has 625 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.950e+05
 Factor NZ  : 1.956e+05 (roughly 4 MBytes of memory)
 Factor Ops : 8.158e+07 (less than 1 second per iteration)
 Threads    : 2

                  Objective                Residual
Iter       Primal      

With these weights, we can now write the image of any basis function $\varphi_i$ under $\mathcal{L}$ as a linear combination of kernels centered at the image points, denoted $\varphi_{y_j}$. But we obviously want to be able to write $\mathcal \varphi_i$ as just a linear combination of the $\varphi_i$. To do this, we will need to write each $\varphi_{y_j}$ itself as a linear combination of the $\varphi_i$.
$$
 \varphi_{y_j} (y) = \sum_{i=1}^m \gamma_i \varphi_i (y)
$$
The weights $\gamma_i$ will be calculated as the integrals of $\varphi_{y_j}$ over a Voronoi tesselation of the basis functions $\varphi_i$, scaled according to weights $c_i$.

In [83]:
X = sample;
Y = S(X);     

In [84]:
Ξ = integrate_phiy(Y, basis_locs, range, 100);

This is now everything we need to compute the matrix $L$.

In [ ]:
L = construct_L(w, Φ, Ξ);